# Analyzing your model with TensorFlow Model Analysis and the What-If Tool

- describe the model here
- what is the dataset https://www.kaggle.com/cfpb/us-consumer-finance-complaints
- what is the model architecture
- how is it saved
- what are the required files

In [3]:
import tensorflow_model_analysis as tfma
import tensorflow as tf

# stop tf warnings going everywhere
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
# model produced by TFX 0.21.1 (install from head at this time)
# 28Feb pipeline
_EVAL_DATA_FILE = '../models/data_tfrecord-00000-of-00001'
_MODEL_DIR = '../models/serving_model_dir/'

Convert the model into the required format for TFMA

In [3]:
eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=_MODEL_DIR, tags=[tf.saved_model.SERVING])

In [4]:
slices = [tfma.slicer.SingleSliceSpec()]

In [5]:
# this takes a minute
# https://www.tensorflow.org/tfx/model_analysis/metrics
tfma_result = tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                                        data_location=_EVAL_DATA_FILE,
                                      eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        #metrics_specs=tfma.metrics.default_binary_classification_specs()
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='FalsePositives'),
                  tfma.MetricConfig(class_name='TruePositives'),
                  tfma.MetricConfig(class_name='FalseNegatives'),
                  tfma.MetricConfig(class_name='TrueNegatives')



        ])]
        ),
                                        file_format='tfrecords', 
                                        slice_spec = slices)

In [6]:
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

Exercise: 

- add in a new metric (e.g. precision, recall)
- read the docs to find available metrics 
- keras metrics: https://www.tensorflow.org/api_docs/python/tf/keras/metrics

ref: https://github.com/tensorflow/model-analysis/blob/9ae6408424f80adf83233f45c2e4d7ba6b86bf06/tensorflow_model_analysis/metrics/metric_specs.py

stretch goal - model performance through time as in here: https://github.com/tensorflow/workshops/blob/master/tfx_labs/Lab_6_Model_Analysis.ipynb

## Model fairness 1 - slicing the data

In [7]:
slices = [tfma.slicer.SingleSliceSpec(),
          tfma.slicer.SingleSliceSpec(columns=['product'])]

In [8]:
# repeated code - turn into a function
tfma_result = tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                                        data_location=_EVAL_DATA_FILE,
                                      eval_config=tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key='consumer_disputed')],
        slicing_specs=[tfma.SlicingSpec(), tfma.SlicingSpec(feature_keys=['product'])],
        #metrics_specs=tfma.metrics.default_binary_classification_specs()
        metrics_specs=[
              tfma.MetricsSpec(metrics=[
                  tfma.MetricConfig(class_name='BinaryAccuracy'),
                  tfma.MetricConfig(class_name='ExampleCount'),
                  tfma.MetricConfig(class_name='FalsePositives'),
                  tfma.MetricConfig(class_name='TruePositives'),
                  tfma.MetricConfig(class_name='FalseNegatives'),
                  tfma.MetricConfig(class_name='TrueNegatives')



        ])]
        ),
                                        file_format='tfrecords', 
                                        slice_spec = slices)

In [11]:
tfma.view.render_slicing_metrics(tfma_result, slicing_spec=slices[1])

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'product:Mortgage', '…

Exercise:

- add in a new slicing column (ref data sample)

## Model fairness 2 - the What-If Tool

In [1]:
from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

In [6]:
eval_data = tf.data.TFRecordDataset(_EVAL_DATA_FILE)

In [7]:
eval_examples = [tf.train.Example.FromString(d.numpy()) for d in eval_data.take(1000)]

In [8]:
model = tf.saved_model.load(export_dir=_MODEL_DIR)

In [9]:
def predict(examples):
    preds = model.signatures['serving_default'](examples=tf.constant([example.SerializeToString() for example in examples]))
    return preds['outputs'].numpy()

In [10]:
config_builder = WitConfigBuilder(eval_examples).set_custom_predict_fn(predict)

In [11]:
WitWidget(config_builder)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…

 Exercise:
 
 - explore the WIT

### Debugging

In [5]:
# works with 2.1
!pip show tensorflow

Name: tensorflow
Version: 2.1.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/i854694/.virtualenvs/aiconf/lib/python3.7/site-packages
Requires: absl-py, opt-einsum, termcolor, wheel, keras-applications, tensorboard, protobuf, google-pasta, astor, keras-preprocessing, gast, six, numpy, grpcio, tensorflow-estimator, scipy, wrapt
Required-by: witwidget, tfx-bsl, tensorflow-serving-api, tensorflow-model-analysis


In [6]:
# works with 0.21.3
!pip show tensorflow_model_analysis

Name: tensorflow-model-analysis
Version: 0.21.3
Summary: A library for analyzing TensorFlow models
Home-page: https://www.tensorflow.org/tfx/model_analysis
Author: Google LLC
Author-email: tensorflow-extended-dev@googlegroups.com
License: Apache 2.0
Location: /Users/i854694/.virtualenvs/aiconf/lib/python3.7/site-packages
Requires: scipy, avro-python3, absl-py, tensorflow, apache-beam, jupyter, tensorflow-metadata, numpy, tfx-bsl, ipywidgets, six, protobuf
Required-by: 


In [13]:
# works with 1.6.0
!pip show witwidget

Name: witwidget
Version: 1.6.0
Summary: What-If Tool jupyter widget
Home-page: https://github.com/pair-code/what-if-tool
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/i854694/.virtualenvs/aiconf/lib/python3.7/site-packages
Requires: tensorflow, ipywidgets, six, google-api-python-client, absl-py, jupyter, oauth2client
Required-by: 


In [12]:
# may need to run this every time
!jupyter nbextension install --py --symlink --sys-prefix witwidget
!jupyter nbextension enable witwidget --py --sys-prefix 

Installing /Users/i854694/.virtualenvs/aiconf/lib/python3.7/site-packages/witwidget/static -> wit-widget
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable witwidget --py --sys-prefix
    
Enabling notebook extension wit-widget/extension...
      - Validating: OK


In [ ]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
  
!jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
  
!jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix

In [ ]:
!pip install widgetsnbextension

In [ ]:
!pip install -U ipywidgets

In [ ]:
!jupyter nbextension list

In [ ]:
!jupyter serverextension list